In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

**Table 4 Panel A**

In [34]:
merged_data = []

# Load datasets
p10umd = pd.read_stata("P10UMD.dta")
factor_ff5 = pd.read_stata("FactorFF5.dta")[['year', 'month', 'mktrf', 'smb', 'hml', 'cma', 'rmw', 'rf', 'yyyymm']]
tsfactor = pd.read_stata("TSFactor.dta")[['year', 'month', 'TSMom', 'yyyymm']]
factor_umd = pd.read_stata("FactorUMD.dta")[['year', 'month', 'umd']]
oos_tsmom = pd.read_stata("oos_tsmom_scs.dta")[['yyyymm', 'tsmom1', 'tsmom2', 'tsmom3', 'tsmom4', 'tsmom5']]

# Merge datasets except for the principal components
merged_data = pd.merge(p10umd, factor_ff5, on=['year', 'month'], how='inner')
merged_data = pd.merge(merged_data, tsfactor, on=['year', 'month'], how='inner')
merged_data = pd.merge(merged_data, factor_umd, on=['year', 'month'], how='inner')

merged_data['year'] = merged_data['year'].astype('int64')
merged_data['month'] = merged_data['month'].astype('int64')

# Merge oos_tsmom and perform necessary operations
oos_tsmom['year'] = oos_tsmom['yyyymm'] // 100
oos_tsmom['month'] = oos_tsmom['yyyymm'] % 100
oos_tsmom = oos_tsmom.drop(columns=['yyyymm'])
oos_tsmom = oos_tsmom.sort_values(by=['year', 'month'], ascending=[True, True])

# Multiply columns by 100 and rename
columns_to_multiply = ['tsmom1', 'tsmom2', 'tsmom3', 'tsmom4', 'tsmom5']
columns_to_rename = ['pctsmom1', 'pctsmom2', 'pctsmom3', 'pctsmom4', 'pctsmom5']
for orig_col, new_col in zip(columns_to_multiply, columns_to_rename):
    oos_tsmom[new_col] = oos_tsmom[orig_col] * 100

# Calculate the excess returns
for i in range(1, 11):
    merged_data[f'ExcessP{i}'] = merged_data[f'p{i}'] - merged_data['rf']

# Portfolio 11 is high-minus-low
merged_data['ExcessP11'] = merged_data['ExcessP10'] - merged_data['ExcessP1']
    
# Create a second dataframe for the Principal Component model, which has less observations
merged_data1 = pd.merge(merged_data, oos_tsmom, on=['year', 'month'], how='inner')

# Also calculate the excess returns for this dataframe
for i in range(1, 11):
    merged_data1[f'ExcessP{i}'] = merged_data1[f'p{i}'] - merged_data1['rf']

merged_data1['ExcessP11'] = merged_data1['ExcessP10'] - merged_data1['ExcessP1']

estimates_list = []

print(merged_data)
merged_data.columns

        p1    p2    p3    p4    p5    p6    p7    p8    p9   p10  ...  \
0     1.93  2.01  3.41  3.03  3.45  2.52  0.10  3.06  2.06  0.70  ...   
1    -1.32  0.49 -1.13 -0.07 -1.32 -1.93 -1.51 -1.08 -1.72 -0.91  ...   
2     5.77  3.02  3.81  3.03  2.58  2.31  1.81  3.74  2.24  6.37  ...   
3     1.52  1.79  1.31  0.50  0.74 -0.48  1.91  0.71  1.01  0.01  ...   
4    -3.05  0.33  0.83 -0.24 -0.32  0.20  0.00  1.79 -0.18  0.04  ...   
..     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
661 -12.06 -9.91 -8.14 -6.61 -3.51 -5.80 -1.72 -0.48  0.91 -0.31  ...   
662   5.32  5.26  5.89  5.16  0.36  3.03  4.44  1.42  0.34 -3.03  ...   
663  -3.92  2.38  4.04  4.50  3.45  4.79  1.92  1.53  1.01 -0.40  ...   
664   3.88  6.00  6.84  5.82  5.18  3.79  4.48  1.69  2.95  3.43  ...   
665  11.06  5.18  3.22  3.69  1.60  2.85  2.42  2.75  2.12  4.60  ...   

     ExcessP2  ExcessP3  ExcessP4  ExcessP5  ExcessP6  ExcessP7  ExcessP8  \
0        1.71      3.11      2.73      3.15   

Index(['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'year',
       'month', 'mktrf', 'smb', 'hml', 'cma', 'rmw', 'rf', 'yyyymm_x', 'TSMom',
       'yyyymm_y', 'umd', 'ExcessP1', 'ExcessP2', 'ExcessP3', 'ExcessP4',
       'ExcessP5', 'ExcessP6', 'ExcessP7', 'ExcessP8', 'ExcessP9', 'ExcessP10',
       'ExcessP11'],
      dtype='object')

In [52]:
estimatesFF5_list = []

# Momentum sorted portfolios with FF5
for i in range(1, 12):
    X = sm.add_constant(merged_data[['mktrf', 'smb', 'hml', 'cma', 'rmw']])
    model = sm.OLS(merged_data[f'ExcessP{i}'], X).fit()
    estimatesFF5_list.append(model)

#for i, est in enumerate(estimatesFF5_list, start=1):
    #print(f"\nRegression Results for Excess Portfolio Returns with FF5 - Portfolio {i}:\n")
    #print(est.summary())

In [56]:
#print("Results for FF5 Model:\n")
resultsFF5_list = []

# Results for FF5
for i, est_FF5 in enumerate(estimatesFF5_list, start=1):
    result_row = {
        'Decile': f'{i}',
        'FF5_Alpha': f'{est_FF5.params["const"]:.2f}',
    }
    t_stat_row = {
        'Decile': f'', 
        'FF5_Alpha': f'({est_FF5.tvalues["const"]:.2f})',
    }
    resultsFF5_list.append(result_row)
    resultsFF5_list.append(t_stat_row)

resultsFF5_df = pd.DataFrame(resultsFF5_list)
#print(resultsFF5_df)

In [58]:
estimatesUMD_list = []

# Momentum sorted portfolios with FF5 + UMD
for i in range(1, 12):
    X = sm.add_constant(merged_data[['mktrf', 'smb', 'hml', 'cma', 'rmw', 'umd']])
    model = sm.OLS(merged_data[f'ExcessP{i}'], X).fit()
    estimatesUMD_list.append(model)

#for i, est in enumerate(estimatesUMD_list, start=1):
    #print(f"\nRegression Results for Excess Portfolio Returns with FF5 + UMD - Portfolio {i}:\n")
    #print(est.summary())

In [60]:
#print("Results for FF5 + UMD Model:\n")
resultsUMD_list = []

# Results for FF5 + UMD
for i, est_umd in enumerate(estimatesUMD_list, start=1):
    result_row = {
        'Decile': f'{i}',
        'UMD_Alpha': f'{est_umd.params["const"]:.2f}',
        'UMD_Coefficient': f'{est_umd.params["umd"]:.2f}'
    }
    t_stat_row = {
        'Decile': f'', 
        'UMD_Alpha': f'({est_umd.tvalues["const"]:.2f})',
        'UMD_Coefficient': f'({est_umd.tvalues["umd"]:.2f})'
    }
    resultsUMD_list.append(result_row)
    resultsUMD_list.append(t_stat_row)

resultsUMD_df = pd.DataFrame(resultsUMD_list)
#print(resultsUMD_df)

In [62]:
estimatesFMOMind_list = []

# Momentum sorted portfolios with FF5 + MOMind
for i in range(1, 12):
    X = sm.add_constant(merged_data[['mktrf', 'smb', 'hml', 'cma', 'rmw', 'TSMom']])
    model = sm.OLS(merged_data[f'ExcessP{i}'], X).fit()
    estimatesFMOMind_list.append(model)

#for i, est in enumerate(estimatesFMOMind_list, start=1):
    #print(f"\nRegression Results for Excess Portfolio Returns with FF5 + FMOMind - Portfolio {i}:\n")
    #print(est.summary())

In [64]:
#print("Results for FF5 + FMOMind Model:\n")
resultsFMOMind_list = []

# Results for FF5 + FMOMind
for i, est_FMOMind in enumerate(estimatesFMOMind_list, start=1):
    result_row = {
        'Decile': f'{i}',
        'FMOMind_Alpha': f'{est_FMOMind.params["const"]:.2f}',
        'FMOMind_Coefficient': f'{est_FMOMind.params["TSMom"]:.2f}'
    }
    t_stat_row = {
        'Decile': f'', 
        'FMOMind_Alpha': f'({est_FMOMind.tvalues["const"]:.2f})',
        'FMOMind_Coefficient': f'({est_FMOMind.tvalues["TSMom"]:.2f})'
    }
    resultsFMOMind_list.append(result_row)
    resultsFMOMind_list.append(t_stat_row)

resultsFMOMind_df = pd.DataFrame(resultsFMOMind_list)
#print(resultsFMOMind_df)

In [66]:
estimatesFMOMpc_list = []

# Momentum sorted portfolios with FF5 + UMD
for i in range(1, 12):
    X = sm.add_constant(merged_data1[['mktrf', 'smb', 'hml', 'cma', 'rmw', 'pctsmom1']])
    model = sm.OLS(merged_data1[f'ExcessP{i}'], X).fit()
    estimatesFMOMpc_list.append(model)

#for i, est in enumerate(estimatesFMOMpc_list, start=1):
    #print(f"\nRegression Results for Excess Portfolio Returns with FF5 + FMOMpc - Portfolio {i}:\n")
    #print(est.summary())

In [68]:
#print("Results for FF5 + FMOMpc Model:\n")
resultsFMOMpc_list = []

# Append results for FF5 + FMOMind
for i, est_FMOMpc in enumerate(estimatesFMOMpc_list, start=1):
    result_row = {
        'Decile': f'{i}',
        'FMOMpc_Alpha': f'{est_FMOMpc.params["const"]:.2f}',
        'FMOMpc_Coefficient': f'{est_FMOMpc.params["pctsmom1"]:.2f}'
    }
    t_stat_row = {
        'Decile': f'', 
        'FMOMpc_Alpha': f'({est_FMOMpc.tvalues["const"]:.2f})',
        'FMOMpc_Coefficient': f'({est_FMOMpc.tvalues["pctsmom1"]:.2f})'
    }
    resultsFMOMpc_list.append(result_row)
    resultsFMOMpc_list.append(t_stat_row)

# Convert the list of results to a DataFrame
resultsFMOMpc_df = pd.DataFrame(resultsFMOMpc_list)

# Display the results DataFrame
#print(resultsFMOMpc_df)

In [13]:
resultsUMD_df = resultsUMD_df.drop(columns=['Decile'])
resultsFMOMind_df = resultsFMOMind_df.drop(columns=['Decile'])
resultsFMOMpc_df = resultsFMOMpc_df.drop(columns=['Decile'])

# Merge the results
merged_results = pd.concat([resultsFF5_df, resultsUMD_df, resultsFMOMind_df, resultsFMOMpc_df], axis=1)

print(merged_results)

   Decile FF5_Alpha UMD_Alpha UMD_Coefficient FMOMind_Alpha  \
0       1     -0.75     -0.10           -0.93         -0.04   
1           (-4.05)   (-0.94)        (-36.59)       (-0.28)   
2       2     -0.35      0.13           -0.70          0.16   
3           (-2.73)    (2.08)        (-46.76)        (1.54)   
4       3     -0.20      0.18           -0.54          0.17   
5           (-1.90)    (2.92)        (-38.35)        (1.93)   
6       4     -0.16      0.07           -0.33          0.12   
7           (-1.93)    (1.20)        (-22.77)        (1.69)   
8       5     -0.16     -0.04           -0.17         -0.02   
9           (-2.45)   (-0.65)        (-12.30)       (-0.39)   
10      6     -0.13     -0.09           -0.05         -0.07   
11          (-2.05)   (-1.46)         (-3.52)       (-1.02)   
12      7     -0.12     -0.16            0.07         -0.14   
13          (-1.94)   (-2.72)          (4.73)       (-2.32)   
14      8      0.04     -0.11            0.22         -

In [14]:
alpha_means_FF5 = []
alpha_means_UMD = []
alpha_means_FMOMind = []
alpha_means_FMOMpc = []

# Calculate the absolute values of the aplhas
for i, est_ff5 in enumerate(estimatesFF5_list, start=1):
    alpha_mean_ff5 = np.abs(est_ff5.params['const']).mean()
    alpha_means_FF5.append(alpha_mean_ff5)

for i, est_umd in enumerate(estimatesUMD_list, start=1):
    alpha_mean_umd = np.abs(est_umd.params['const']).mean()
    alpha_means_UMD.append(alpha_mean_umd)

for i, est_fmomind in enumerate(estimatesFMOMind_list, start=1):
    alpha_mean_fmomind = np.abs(est_fmomind.params['const']).mean()
    alpha_means_FMOMind.append(alpha_mean_fmomind)

for i, est_fmompc in enumerate(estimatesFMOMpc_list, start=1):
    alpha_mean_fmompc = np.abs(est_fmompc.params['const']).mean()
    alpha_means_FMOMpc.append(alpha_mean_fmompc)

# Calculate the absolute mean alphas for the different models except for the winners-losers portfolio
Avg_alpha_FF5 = np.mean(alpha_means_FF5[:-1])
Avg_alpha_UMD = np.mean(alpha_means_UMD[:-1])
Avg_alpha_FMOMind = np.mean(alpha_means_FMOMind[:-1])
Avg_alpha_FMOMpc = np.mean(alpha_means_FMOMpc[:-1])

# Create a DataFrame for average alphas
avg_alphas_df = pd.DataFrame({
    'Model': ['FF5', 'UMD', 'FMOMind', 'FMOMpc'],
    'Avg_alpha': [Avg_alpha_FF5, Avg_alpha_UMD, Avg_alpha_FMOMind, Avg_alpha_FMOMpc]
})

print(avg_alphas_df)

     Model  Avg_alpha
0      FF5   0.257050
1      UMD   0.120639
2  FMOMind   0.108118
3   FMOMpc   0.098437


**Panel B**

In [48]:
# Assuming 'UMD' is your dependent variable, and 'mktrf', 'smb', 'hml', 'cma', 'rmw' are your factors
independent_vars_ff5 = ['mktrf', 'smb', 'hml', 'cma', 'rmw']

# Augment the FF5 model with different subsets of PC factors
independent_vars_pctsmom1 = ['mktrf', 'smb', 'hml', 'cma', 'rmw', 'pctsmom1']
independent_vars_pctsmom2 = ['mktrf', 'smb', 'hml', 'cma', 'rmw', 'pctsmom2']
independent_vars_pctsmom3 = ['mktrf', 'smb', 'hml', 'cma', 'rmw', 'pctsmom3']
independent_vars_pctsmom4 = ['mktrf', 'smb', 'hml', 'cma', 'rmw', 'pctsmom4']
independent_vars_pctsmom5 = ['mktrf', 'smb', 'hml', 'cma', 'rmw', 'pctsmom5']

# Create a list to store the results
results_list = []

# Fit the FF5 model
X_ff5 = sm.add_constant(merged_data1[independent_vars_ff5])
model_ff5 = sm.OLS(merged_data1['umd'], X_ff5)
results_ff5 = model_ff5.fit()

# Record the results for the FF5 model (only alpha)
results_list.append({
    'Model': 'FF5',
    'Alpha': results_ff5.params['const'],
    'Alpha T-stat': results_ff5.tvalues['const'],
    'FMom Slope':' ',
    'FMom Slope T-stat':' ',
    'R-squared_adj': results_ff5.rsquared_adj
})

# Fit models with different subsets of PC factors
for subset_vars in [independent_vars_pctsmom1, independent_vars_pctsmom2, independent_vars_pctsmom3, independent_vars_pctsmom4, independent_vars_pctsmom5]:
    X_subset = sm.add_constant(merged_data1[subset_vars])
    model_subset = sm.OLS(merged_data1['umd'], X_subset)
    results_subset = model_subset.fit()

    # Record the results for each subset, including the alpha and slope for pctsmom factor
    results_list.append({
        'Model': ' + '.join(subset_vars[-1:]),
        'Alpha': results_subset.params['const'],
        'Alpha T-stat': results_subset.tvalues['const'],
        'FMom Slope': results_subset.params[subset_vars[-1:][0]],
        'FMom Slope T-stat': results_subset.tvalues[subset_vars[-1:][0]],
        'R-squared_adj': results_subset.rsquared_adj
    })

# Create a DataFrame from the list of results
results_df = pd.DataFrame(results_list)

# Display the results
print(results_df)

      Model     Alpha  Alpha T-stat FMom Slope FMom Slope T-stat  \
0       FF5  0.627999      3.400518                                
1  pctsmom1 -0.091454     -0.594911   3.899118         17.517987   
2  pctsmom2  0.362050      1.994262   2.045442          6.880018   
3  pctsmom3  0.284387      1.611649   3.147788          9.093304   
4  pctsmom4  0.345101      2.035411   3.033553         10.932782   
5  pctsmom5  0.399247      2.349366    2.56848         10.566864   

   R-squared_adj  
0       0.106322  
1       0.424966  
2       0.175528  
3       0.221525  
4       0.264293  
5       0.255559  
